In [1]:
from pymongo import MongoClient, DESCENDING
from pprint import pprint
import re
import pymongo
import json
import os
import pandas as pd
from bson.json_util import dumps

In [2]:
# Instantiate mongo client
myclient = pymongo.MongoClient("mongodb://localhost:27017/")

In [3]:
# List databases
for db in myclient.list_databases():
    print(db)

{'name': 'admin', 'sizeOnDisk': 40960, 'empty': False}
{'name': 'config', 'sizeOnDisk': 61440, 'empty': False}
{'name': 'local', 'sizeOnDisk': 73728, 'empty': False}
{'name': 'nyt_db', 'sizeOnDisk': 168505344, 'empty': False}


## ========= CREATE DATABASE =========

In [132]:
## !!!! Drop database
# myclient.drop_database('nyt_db')

In [4]:
# Create database
nyt_db = myclient["nyt_db"]

# Database collection
nyt_articles_coll = nyt_db["nyt_articles_coll"]

In [135]:
## Read json files and insert in MongoDB

# !! This files are not working. Are excluded
# NYT_2021_11_test.json
# NYT_2021_1_test.json
# NYT_2021_7_test.json
# NYT_2022_5_test.json
# NYT_2022_7_test.json
# NYT_2022_8_test.json

# Read data directory
directory = './nyt_data'

# Iterate over files in the directory
for filename in os.listdir(directory):
    json_file = os.path.join(directory, filename)

    # Jump over this file in my pc ;-)
    if 'desktop.ini' in json_file:
        continue
    
    # checking if it is a file
    if os.path.isfile(json_file):
        print(json_file)
    
    # Read one json file
    with open(json_file, encoding="utf8") as file:
        file_data = json.load(file)

    # Insert in the collection
    nyt_articles_coll.insert_many(file_data["response"]["docs"])


./nyt_data\NYT_2020_10_test.json
./nyt_data\NYT_2020_11_test.json
./nyt_data\NYT_2020_12_test.json
./nyt_data\NYT_2020_1_test.json
./nyt_data\NYT_2020_2_test.json
./nyt_data\NYT_2020_3_test.json
./nyt_data\NYT_2020_4_test.json
./nyt_data\NYT_2020_5_test.json
./nyt_data\NYT_2020_6_test.json
./nyt_data\NYT_2020_7_test.json
./nyt_data\NYT_2020_8_test.json
./nyt_data\NYT_2020_9_test.json
./nyt_data\NYT_2021_10_test.json
./nyt_data\NYT_2021_12_test.json
./nyt_data\NYT_2021_2_test.json
./nyt_data\NYT_2021_3_test.json
./nyt_data\NYT_2021_4_test.json
./nyt_data\NYT_2021_5_test.json
./nyt_data\NYT_2021_6_test.json
./nyt_data\NYT_2021_8_test.json
./nyt_data\NYT_2021_9_test.json
./nyt_data\NYT_2022_10_test.json
./nyt_data\NYT_2022_11_test.json
./nyt_data\NYT_2022_1_test.json
./nyt_data\NYT_2022_2_test.json
./nyt_data\NYT_2022_3_test.json
./nyt_data\NYT_2022_4_test.json
./nyt_data\NYT_2022_6_test.json
./nyt_data\NYT_2022_9_test.json


In [5]:
# Check MongoDb
# Print the total number of articles
# Or use MongoDB Compass

# == QUERY ==
total_articles = nyt_articles_coll.count_documents({})

# == PRINT RESULTS ==
print("The number of articles in this collections is", total_articles)   #  126465

The number of articles in this collections is 126465


## ========= CREATE DataFrame and Save to csv =========

In [137]:
# ===========================================
# CREATE THE csv FILE 
# with PANDAS
# ===========================================

import pandas as pd

# Now creating a Cursor instance using find() function
cursor = nyt_articles_coll.aggregate(
    [
        {
            "$project": {
                '_id': 1,
                'abstract' : 1,
                'web_url' : 1,
                'snippet' : 1,
                'lead_paragraph' : 1,
                'print_section' : 1,
                'print_page' : 1,
                'a_source' : 1,
                'headline_main' : '$headline.main',
                'headline_print_headline' : '$headline.print_headline',
                'pub_date' : 1,
                'document_type' : 1,
                'news_desk' : 1,
                'section_name' : 1,
                'byline_original' : '$byline.original',
                'byline_organization' : '$byline.organization',
                'type_of_material' : 1,
                'word_count' : 1                
            }
        }
    ]
)

# Expand the cursor and construct the DataFrame 'articles'
df_raw =  pd.DataFrame(list(cursor))
df_raw.to_csv('./nyt_csv_files/nyt_data_raw.csv', index=False)

## ========= QUERIES to TEST =========

In [138]:
## Print field of 10 articles

# == CONFIGURATION ==
# Configure which field and word (exact match, case insensitive) 
# to search for counting the articles
# !! Nested fields are separed with (.) dot
# !! i.e.: query_field = 'headline.print_headline'
query_field = 'pub_date'


# == QUERY ==
query_results = nyt_articles_coll.find(
                    projection = {
                        query_field : 1,
                        '_id' : 0
                    }
                ).limit(5)
          

# == PRINT RESULTS ==

# = Normal
# pprint(list(query_results))


# = With Pandas
# Converting cursor to the list of dictionaries
list_cur = list(query_results)
  
# Converting to the DataFrame
df = pd.DataFrame(list_cur)
  
# Printing the df to console
display(df.head(50))


,pub_date
0,2020-10-01T00:05:51+0000
1,2020-10-01T00:30:45+0000
2,2020-10-01T00:43:28+0000
3,2020-10-01T00:45:04+0000
4,2020-10-01T00:45:17+0000


In [8]:
## Count articles with a word or phrase

# == CONFIGURATION ==
# Configure which field and word (exact match, case insensitive) 
# to search for counting the articles
# !! Nested fields are separed with (.) dot
# !! i.e.: query_field = 'headline.print_headline'
query_field = 'byline_original'
query_words = 'Collins'


# == QUERY ==
# SELECT COUNT(*) FROM articles 
# WHERE query_field lIKE '%query_words%'
nr_with = nyt_articles_coll.count_documents(
                filter = { 
                            query_field : {
                                            '$regex': query_words,
                                            '$options' : 'i'
                            }
                } 
            )

# SELECT COUNT(*) FROM articles 
# WHERE query_field NOT lIKE '%query_words%'
nr_without = nyt_articles_coll.count_documents( 
                filter = { 
                            query_field : { 
                                            '$not': {
                                                '$regex': query_words,
                                                '$options' : 'i'
                                            }
                            }
                } 
            )

# COUNT ALL Articles (documents)
total_articles = nyt_articles_coll.count_documents({})


# == PRINT RESULTS ==
print(  'Nr Articles WITH word(s)    "' + query_words + '"  IN "' + query_field + '" =', nr_with, 
      '\nNr Articles WITHOUT word(s) "' + query_words + "'  IN '" + query_field + '" =', nr_without,
      '\nCHECK: Nr Total =', total_articles, '  |  Sum =', (nr_with + nr_without) )


Nr Articles WITH word(s)    "Collins"  IN "byline_original" = 0 
Nr Articles WITHOUT word(s) "Collins'  IN 'byline_original" = 126465 
CHECK: Nr Total = 126465   |  Sum = 126465


In [140]:
## FIND articles with a word or phrase

# == CONFIGURATION ==
# Configure which field and word (exact match, case insensitive) 
# to search for counting the articles
# !! Nested fields are separed with (.) dot
# !! i.e.: query_field = 'headline.print_headline'
query_field = 'abstract'
query_words = 'covid'
show_filed = 'headline.main'
# show_filed = 'abstract'


# == QUERY ==
# SELECT show_filed FROM articles 
# WHERE query_field lIKE '%query_words%'
query_results = nyt_articles_coll.find(
                    filter = { 
                                query_field : {
                                                '$regex' : query_words,
                                                '$options' : 'i'
                                }
                    },
                    projection = {
                                    show_filed : 1, 
                                    '_id' : 0
                    }
                )


# == PRINT RESULTS ==
# Print 'explanation'
print("=============== SELECT ===============", \
      "\nSELECT", show_filed,  "FROM articles", \
      "\nWHERE" , query_field, "LIKE '%" + query_words + "%'", \
      "\n======================================\n\n"
     )


# = Normal
# pprint(list(query_results))


# = With Pandas
# Converting cursor to the list of dictionaries
list_cur = list(query_results)
  
# Converting to the DataFrame
df = pd.DataFrame(list_cur)
  
# Printing the df to console
display(df.head(5))

=============== SELECT =============== 
SELECT headline.main FROM articles 
WHERE abstract LIKE '%covid%' 




,headline
0,{'main': 'Tenants Facing Eviction Over Covid-1...
1,{'main': 'Where Does America Sit in a Pandemic...
2,"{'main': 'Grace Meo, Skilled Seamstress and Se..."
3,{'main': 'Trump Has the Coronavirus. What Risk...
4,{'main': 'Video Shows Trump Tossing Hats to Cr...


In [143]:
## CREATE TEXT INDIXES to SEARCH
# Not sure how to use this effectively
# ToDo Optional

# resp = nyt_articles_coll.create_index(
#     [
#         ('abstract', 'text'),
#         ('snippet', 'text'),
#         ('lead_paragraph', 'text')
#     ],
#     name='abstract_snippet_leadparagraph_index'
# )

# # Drop index
# nyt_articles_coll.drop_index('abstract_text_snippet_text_lead_paragraph_text')

# # Get indexes
# pprint(nyt_articles_coll.index_information())

## FULL SEARCH with TEXT

# # 'word1 word2' ==> word1 OR word2
# # '\"\word1 word2\"' ==> "word1 word2" ==> exact phrase in this order

# query_words = 'covid corona omicron'   # word1 OR word2

# query_results = nyt_articles_coll.find(
#                     filter = {
#                         '$text' : {
#                             '$search' : query_words
#                         }
#                     },
#                 )


In [141]:
## Group and Count per field value

# == CONFIGURATION ==
# SELECT COUNT(*) FROM articles
# GROUP BY query_field
# ORDER COUNT(*) DESC
# LIMIT 10
query_field = 'news_desk'


# == QUERY ==
pipeline = [ 
    {'$group':
        {
            '_id': '$'+query_field, 
            'NrArticles':  {'$sum': 1 } 
        }
    },
    {'$sort': {'NrArticles': -1} },
    {'$limit': 50 }
]

query_results = nyt_articles_coll.aggregate(pipeline)


# == PRINT RESULTS ==
# == Normal
# pprint(list(query_results))


# == With Pandas
# Converting cursor to the list of dictionaries
list_cur = list(query_results)
  
# Converting to the DataFrame
df = pd.DataFrame(list_cur)
  
# Printing the df to console
display(df.head(5))

,_id,NrArticles
0,Foreign,12206
1,Business,9667
2,OpEd,8294
3,Culture,8110
4,Washington,7355


In [146]:
## Create an AUTOMATIC distribution. Like an Histogram
# Nr of Articles per Word Count Automatic-Range

# == QUERY ==
pipeline = [
    {
        '$bucketAuto' : {
             'groupBy': '$word_count',
             'buckets': 10
         }
    }
]

query_results = nyt_articles_coll.aggregate(pipeline)


# == PRINT RESULTS ==
# == Normal
# pprint(list(query_results))


# == With Pandas
# Converting cursor to the list of dictionaries
list_cur = list(query_results)
  
# Converting to the DataFrame
df = pd.DataFrame(list_cur)
  
# Printing the df to console
display(df.head(10))

# == TEST RESULTS ==
query_results = nyt_articles_coll.count_documents( 
    filter = {
        'word_count' : { 
             '$gte' : 1610
         }
    }
)

print(query_results)

,_id,count
0,"{'min': 0, 'max': 71}",12655
1,"{'min': 71, 'max': 360}",12662
2,"{'min': 360, 'max': 562}",12686
3,"{'min': 562, 'max': 758}",12656
4,"{'min': 758, 'max': 925}",12649
5,"{'min': 925, 'max': 1084}",12686
6,"{'min': 1084, 'max': 1227}",12679
7,"{'min': 1227, 'max': 1377}",12670
8,"{'min': 1377, 'max': 1610}",12649
9,"{'min': 1610, 'max': 20573}",12473


12473


In [147]:
## Create a MANUAL distribution. Like an Histogram
# Nr of Articles per Word Count Manual-Range
# i.e.: [0-500) = 34219   // O included - 500 excluded

# == QUERY ==
pipeline = [
    {
        '$bucket' : {
            'groupBy' : '$word_count',
            'boundaries' : [ 0,500,1000,1500,2000,2500,3000,3500,4000,4500, 5000, 5500 ],
            'default' : 'Above 5500',
            'output' : {
                'count' : { '$sum' : 1 }
            }
        }
    }
]

query_results = nyt_articles_coll.aggregate(pipeline)


# == PRINT RESULTS ==
# == Normal
# pprint(list(query_results))


# == With Pandas
# Converting cursor to the list of dictionaries
list_cur = list(query_results)
  
# Converting to the DataFrame
df = pd.DataFrame(list_cur)
  
# Printing the df to console
display(df.head(13))


,_id,count
0,0,34219
1,500,34964
2,1000,39652
3,1500,12799
4,2000,2146
5,2500,1020
6,3000,403
7,3500,253
8,4000,193
9,4500,176


In [148]:
## Query the UNIQUE DISTINCT values of a field

# Configure the field you want to search unique
query_field = 'section_name'  # 60 results has NULL
query_field = 'news_desk'     # 95 results has NULL
query_field = 'print_section'     # 30 results  NOT NULL
query_field = 'source'     # 1 resultado NOT NULL
query_field = 'document_type'     # 4 results NOT NULL
query_field = 'type_of_material'     # 21 results NOT NULL


name_result_query_field = 'unique_' + query_field

pipeline =[
  {
    '$unwind': {
      'path': '$' + query_field,
      'preserveNullAndEmptyArrays': True
    }
  },
  {
    '$group': {
      '_id': 'null',
      name_result_query_field: { '$addToSet': '$' + query_field }
    }
  }
]

query_results = nyt_articles_coll.aggregate(pipeline)


# == PRINT RESULTS ==
# == Normal
pprint(list(query_results))


[{'_id': 'null',
  'unique_type_of_material': ['',
                              'Biography',
                              'List',
                              'Schedule',
                              'News Analysis',
                              'News',
                              'Obituary (Obit)',
                              'Text',
                              "Editors' Note",
                              'Letter',
                              'Editorial',
                              'Review',
                              'Interactive Feature',
                              'Question',
                              'Slideshow',
                              'Op-Ed',
                              'Quote',
                              'briefing',
                              'Video',
                              'Correction',
                              'An Appraisal']}]


#=======================================================#

In [ ]:
################################################
## CREATE THE JSON FILE for Elastic Search
## Did NOT work like this
# from bson.json_util import dumps

# # Now creating a Cursor instance using find() function
# cursor = nyt_articles_coll.find(
#     { },
#     {
#         'abstract' : 1,
#         'snippet' : 1,
#         'lead_paragraph' : 1,
#         'headline.main' : 1,
#         'pub_date' : 1,
#         'document_type' : 1,
#         'news_desk' : 1,
#         'section_name' : 1,
#         'type_of_material' : 1
#     }
# )


# # Converting cursor to the list of dictionaries
# list_cur = list(cursor)

# # Converting to the JSON
# json_data = dumps(list_cur, ensure_ascii=False, indent = 2)

   
# # Writing data to file data.json
# with open('./nyt_data_for_elasticsearch.json', 'w', encoding="utf8") as file:
#     file.write(json_data)

In [149]:
query_results = nyt_articles_coll.aggregate(
    [
        {
            "$project": {
                "_id": 0,
                'pub_date' : 1,
                'abstract' : 1,
                'snippet' : 1,
                'lead_paragraph' : 1,
                'headlinem': '$headline.main',
                'document_type' : 1,
                'news_desk' : 1,
                'section_name' : 1,
                'type_of_material' : 1
            }
        }
    ]
)


# == PRINT RESULTS ==
# == Normal
# pprint(list(query_results))


# == With Pandas
# Converting cursor to the list of dictionaries
list_cur = list(query_results)
  
# Converting to the DataFrame
df = pd.DataFrame(list_cur)
  
# Printing the df to console
display(df.head(10))

,abstract,snippet,lead_paragraph,pub_date,document_type,news_desk,section_name,type_of_material,headlinem
0,Our president as a terrible toddler.,Our president as a terrible toddler.,When the nation looks back on the presidential...,2020-10-01T00:05:51+0000,article,OpEd,Opinion,Op-Ed,Anyone Else Want to See Trump ‘Shut Up’?
1,The Lakers fell into a deficit early but fough...,The Lakers fell into a deficit early but fough...,LeBron James is not accustomed to Game 1s like...,2020-10-01T00:30:45+0000,article,Sports,Sports,News,How the Lakers Beat the Heat in Game 1 of the ...
2,"Instead of condemning violent groups, the pres...","Instead of condemning violent groups, the pres...",President Trump didn’t hurt Joe Biden in Tuesd...,2020-10-01T00:43:28+0000,article,OpEd,Opinion,Op-Ed,Trump Calls on Extremists to ‘Stand By’
3,Facebook on Wednesday said it would prohibit t...,,Facebook on Wednesday said it would prohibit t...,2020-10-01T00:45:04+0000,article,Politics,U.S.,News,Facebook will forbid ads that undermine the le...
4,If the Democratic Party claims to value Black ...,If the Democratic Party claims to value Black ...,"At the end of the Civil War in 1865, Mississip...",2020-10-01T00:45:17+0000,article,OpEd,Opinion,Op-Ed,"Can Mike Espy Make History, Again?"
5,The majority of taxpayers can’t afford the exp...,The majority of taxpayers can’t afford the exp...,We don’t know if Donald Trump’s recent federal...,2020-10-01T00:54:19+0000,article,OpEd,Opinion,Op-Ed,Trump’s Tax Avoidance Is a Tax on the Rest of Us
6,The exchange resumed trading on Friday after a...,The exchange resumed trading on Friday after a...,TOKYO — The Tokyo Stock Exchange shut down for...,2020-10-01T01:32:53+0000,article,Business,Business Day,News,"Tokyo Stock Market Halts Trading for a Day, Ci..."
7,A slide show including some of our most popula...,A slide show including some of our most popula...,A slide show including some of our most popula...,2020-10-01T01:50:28+0000,multimedia,The Learning Network,The Learning Network,Slideshow,Images From Four More Years of ‘What’s Going O...
8,Adam Fromm is on the line.,Adam Fromm is on the line.,THURSDAY PUZZLE — Devious. But a great “aha” m...,2020-10-01T02:00:05+0000,article,Games,Crosswords & Games,News,In Which Rikishi Wear Mawashi
9,"Quotation of the Day for Thursday, October 1, ...","Quotation of the Day for Thursday, October 1, ...",“The country we have looked to for leadership ...,2020-10-01T02:12:33+0000,article,Summary,Today’s Paper,Quote,"Quotation of the Day: Insults, Bluster and a S..."


### ====================================